In [ ]:
!pip install numpy transformers sentencepiece datasets trax

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 637.9/637.9 kB 45.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 54.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━

In [ ]:
#Imports
import numpy as np
from google.colab import drive
from datasets import load_dataset
from transformers import AdamW, AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import get_linear_schedule_with_warmup
import random
import numpy as np

import trax
from trax import layers as tl
from trax.fastmath import numpy as fastnp
from trax.supervised import training

In [ ]:
drive.mount('/content/drive')

/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Mounted at /content/drive


In [ ]:
path_to_file = '/content/drive/My Drive/Mydata/dev.txt'
model_repo = 'google/mt5-base'

In [ ]:
# Load the Opus-100 corpus for English-Arabic translation
opus100 = load_dataset("opus100", "ar-en")

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset opus100 downloaded and prepared to /root/.cache/huggingface/datasets/opus100/ar-en/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

Whatever comes after is a test

In [ ]:



# Load the BERT tokenizer for Arabic
#tokenizer = AutoTokenizer.from_pretrained("asafaya/bert-base-arabic")

# Tokenize the input and output sentences
arabic_target = []
english_source = []
#print(opus100['train'][1]['translation']['ar'])
size = opus100['train'].num_rows
for i in range(size) :
  arabic_target.append(opus100['train'][i]['translation']['ar'])
  english_source.append(opus100['train'][i]['translation']['en'])

    # Print the tokenized input and output sentences
    #print(input_tokens)
    #print(output_tokens)



In [ ]:
print(len(english_source))

1000000


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_repo)

/usr/local/lib/python3.9/dist-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
import random as rnd
rnd.seed(32)

In [ ]:
def encode_str(text, tokenizer, seq_len):

  # Tokenize and add special tokens
  input_ids = tokenizer.encode(
      text = text,
      return_tensors = 'np',
      padding = 'max_length',
      truncation = True,
      max_length = seq_len)

  return input_ids[0]


#def decode_out()

def data_gen(source , target , tokenizer , batch_size , shuffle = True,pad = 0):
  num_lines = len(source)
  lines_index = [*range(num_lines)]
  if shuffle:
        rnd.shuffle(lines_index)
  index = 0
  x_selection = [0]*batch_size
  y_selection = [0]*batch_size
  while True:
        x_selection = [0]*batch_size
        y_selection = [0]*batch_size
        max_len = 0
        for i in range(batch_size):
            if index >= num_lines:
                index = 0
                if shuffle:
                    rnd.shuffle(lines_index)
            x_selection[i] = source[lines_index[index]]

            y_selection[i] = target[lines_index[index]]

            maxlenselection = max(len(target[lines_index[index]]),len(source[lines_index[index]]))
            if maxlenselection > max_len:
                max_len = maxlenselection
            index +=1
        X = np.full((batch_size, max_len), pad)
        Y = np.full((batch_size, max_len), pad)
        for i in range(batch_size):
            X[i] = encode_str(x_selection[i], tokenizer, max_len)
            Y[i] = encode_str(y_selection[i], tokenizer, max_len)
        yield ((X,Y))

In [ ]:
input_ids = tokenizer.encode(
      text = 'this is a test!',
      return_tensors = 'np')
print(input_ids)

[[ 714  339  259  262 2978  309    1]]


In [ ]:
batch_size = 16
data_gen = data_gen(english_source , arabic_target , tokenizer , batch_size )


In [ ]:
random_text = 'أنا أحاول أن أتعلم اللغة العربية، ولكن النحو والصرف يثيران فيّ الكثير من الصعوبات'
token_ids = encode_str(random_text , tokenizer , 100)
tokens = tokenizer.convert_ids_to_tokens(token_ids)
print(tokens)


['▁', 'أنا', '▁أ', 'حاول', '▁أن', '▁أ', 'تعلم', '▁ال', 'لغ', 'ة', '▁العربية', '،', '▁و', 'لكن', '▁ال', 'نحو', '▁وال', 'صرف', '▁', 'يث', 'يران', '▁في', 'ّ', '▁ال', 'كثير', '▁من', '▁ال', 'صع', 'وبات', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>']


In [ ]:
data_batch = next(data_gen)
print('Input :', data_batch[0])
print('Output :', data_batch[1])

Input : [[  1662    277    380 ...      0      0      0]
 [118319    334    309 ...      0      0      0]
 [  1662    259  58620 ...      0      0      0]
 ...
 [ 27307   4065    514 ...      0      0      0]
 [  3398   1469    375 ...      0      0      0]
 [ 39579   1459    336 ...      0      0      0]]
Output : [[ 18765   5323 120730 ...      0      0      0]
 [   259  40694   4271 ...      0      0      0]
 [   259    264   1377 ...      0      0      0]
 ...
 [ 24592 163145   8092 ...      0      0      0]
 [  8403   5623    259 ...      0      0      0]
 [  3510    905    572 ...      0      0      0]]


In [ ]:
opus100

DatasetDict({
    test: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
    train: Dataset({
        features: ['translation'],
        num_rows: 1000000
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 2000
    })
})

In [ ]:
arabic_target_eval = []
english_source_eval = []
#print(opus100['train'][1]['translation']['ar'])
size = opus100['test'].num_rows
for i in range(size) :
  arabic_target_eval.append(opus100['test'][i]['translation']['ar'])
  english_source_eval.append(opus100['test'][i]['translation']['en'])

In [ ]:
print(len(english_source_eval))

2000


In [ ]:
def encode2str(text, tokenizer):
  input_ids = tokenizer.encode(
      text = text,
      return_tensors = 'np')
  return input_ids[0]
def data_gen2(source , target , tokenizer , shuffle = True,pad = 0):
  num_lines = len(source)
  lines_index = [*range(num_lines)]
  if shuffle:
      rnd.shuffle(lines_index)
  index = 0
  while True:
      if index >= num_lines:
          index = 0
          if shuffle:
              rnd.shuffle(lines_index)
      x_selection = source[lines_index[index]]

      y_selection = target[lines_index[index]]

      index +=1
      X = encode2str(x_selection, tokenizer)
      Y = encode2str(y_selection, tokenizer)
      yield ((X,Y))

def eval_gen2(source , target , tokenizer , shuffle = True,pad = 0):
  num_lines = len(source)
  lines_index = [*range(num_lines)]
  if shuffle:
      rnd.shuffle(lines_index)
  index = 0
  while True:
      if index >= num_lines:
          index = 0
          if shuffle:
              rnd.shuffle(lines_index)
      x_selection = source[lines_index[index]]

      y_selection = target[lines_index[index]]

      index +=1
      X = encode2str(x_selection, tokenizer)
      Y = encode2str(y_selection, tokenizer)
      yield ((X,Y))

In [ ]:
def eval_gen(source , target , tokenizer , batch_size , shuffle = True,pad = 0):
  num_lines = len(source)
  lines_index = [*range(num_lines)]
  if shuffle:
        rnd.shuffle(lines_index)
  index = 0
  x_selection = [0]*batch_size
  y_selection = [0]*batch_size
  while True:
        x_selection = [0]*batch_size
        y_selection = [0]*batch_size
        max_len = 0
        for i in range(batch_size):
            if index >= num_lines:
                index = 0
                if shuffle:
                    rnd.shuffle(lines_index)
            x_selection[i] = source[lines_index[index]]

            y_selection[i] = target[lines_index[index]]

            maxlenselection = max(len(target[lines_index[index]]),len(source[lines_index[index]]))
            if maxlenselection > max_len:
                max_len = maxlenselection
            index +=1
        X = np.full((batch_size, max_len), pad)
        Y = np.full((batch_size, max_len), pad)
        for i in range(batch_size):
            X[i] = encode_str(x_selection[i], tokenizer, max_len)
            Y[i] = encode_str(y_selection[i], tokenizer, max_len)
        yield ((X,Y))

In [ ]:
data_gen = data_gen2(english_source , arabic_target , tokenizer )
#data_gen_eval = eval_gen(english_source_eval , arabic_target_eval , tokenizer , batch_size )
data_gen_eval = eval_gen2(english_source_eval , arabic_target_eval , tokenizer)

train_input, train_target = next(data_gen)

In [ ]:
filtered_train_stream = trax.data.FilterByLength(
    max_length=512, length_keys=[0, 1])(data_gen)

filtered_eval_stream = trax.data.FilterByLength(
    max_length=512, length_keys=[0, 1])(data_gen_eval)

In [ ]:
train_input, train_target = next(filtered_train_stream)
print(f'Single tokenized example input:', 'red' , train_input)
print(f'Single tokenized example target:', 'red', train_target)
print("input_batch shape: ", train_input.shape)
print("target_batch shape: ", train_target.shape)

Single tokenized example input: red [   486    259    265 125666    260      1]
Single tokenized example target: red [   259 125944   5687    260      1]
input_batch shape:  (6,)
target_batch shape:  (5,)


In [ ]:
boundaries =  [8,   16,  32, 64, 128, 256, 512]
batch_sizes = [256, 128, 64, 32, 16,    8,   4,  2]

# Create the generators.
train_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(filtered_train_stream)

In [ ]:
eval_batch_stream = trax.data.BucketByLength(
    boundaries, batch_sizes,
    length_keys=[0, 1]  # As before: count inputs and targets to length.
)(filtered_eval_stream)

# Add masking for the padding (0s).
train_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(train_batch_stream)
eval_batch_stream = trax.data.AddLossWeights(id_to_mask=0)(eval_batch_stream)

In [ ]:
input_batch, target_batch, mask_batch = next(train_batch_stream)

# let's see the data type of a batch
print("input_batch data type: ", type(input_batch))
print("target_batch data type: ", type(target_batch))

# let's see the shape of this particular batch (batch length, sentence length)
print("input_batch shape: ", input_batch.shape)
print("target_batch shape: ", target_batch.shape)

input_batch data type:  <class 'numpy.ndarray'>
target_batch data type:  <class 'numpy.ndarray'>
input_batch shape:  (16, 128)
target_batch shape:  (16, 128)


In [ ]:
VOCAB_SIZE = tokenizer.vocab_size
VOCAB_FILE = tokenizer.vocab_file


In [ ]:
from termcolor import colored
index = random.randrange(len(train_input))
print(colored('THIS IS THE ENGLISH SENTENCE: \n', 'red'), tokenizer.convert_ids_to_tokens(input_batch[index]), '\n')
print(colored('THIS IS THE TOKENIZED VERSION OF THE ENGLISH SENTENCE: \n ', 'red'), input_batch[index], '\n')
print(colored('THIS IS THE GERMAN TRANSLATION: \n', 'red'), tokenizer.convert_ids_to_tokens(target_batch[index]), '\n')
print(colored('THIS IS THE TOKENIZED VERSION OF THE GERMAN TRANSLATION: \n', 'red'), target_batch[index], '\n')

print(tokenizer.decode(target_batch[index], add_special_tokens=True))

THIS IS THE ENGLISH SENTENCE: 
 ['▁Communication', '▁No', '.', '▁511', '/1992', '.', '</s>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>', '<pad>'

In [ ]:
def input_encoder_fn(input_vocab_size, d_model, n_encoder_layers):

    input_encoder = tl.Serial(
        tl.Embedding(vocab_size= input_vocab_size,d_feature=d_model),
        [tl.LSTM(n_units = d_model) for _ in range(n_encoder_layers)]
    )

    return input_encoder

def pre_attention_decoder_fn(mode, target_vocab_size, d_model):

    pre_attention_decoder = tl.Serial(
        tl.ShiftRight(),
        tl.Embedding(vocab_size = target_vocab_size , d_feature = d_model ),
        tl.LSTM(n_units = d_model)
    )
    return pre_attention_decoder


def prepare_attention_input(encoder_activations, decoder_activations, inputs):

    keys = encoder_activations
    values = encoder_activations
    queries = decoder_activations
    mask = inputs > 0
    mask = fastnp.reshape(mask, (mask.shape[0], 1, 1, mask.shape[1]))
    mask = mask + fastnp.zeros((1, 1, decoder_activations.shape[1], 1))


    return queries, keys, values, mask

def NMTAttn(input_vocab_size=VOCAB_SIZE, target_vocab_size=VOCAB_SIZE,d_model=256,n_encoder_layers=2,n_decoder_layers=2,n_attention_heads=4,attention_dropout=0.0,mode='train'):

    input_encoder =  input_encoder_fn(input_vocab_size, d_model, n_encoder_layers)
    pre_attention_decoder = pre_attention_decoder_fn(mode, target_vocab_size, d_model)
    model = tl.Serial(
      tl.Select([0,1,0,1]),
      tl.Parallel(input_encoder,pre_attention_decoder),
      tl.Fn('PrepareAttentionInput', prepare_attention_input, n_out=4),
      tl.Residual(tl.AttentionQKV(d_model, n_heads=n_attention_heads, dropout=attention_dropout, mode=mode)),
      tl.Select([0,2]),
      [tl.LSTM(n_units = d_model) for _ in range(n_decoder_layers)],
      tl.Dense(target_vocab_size),
      tl.LogSoftmax()
    )

    return model

In [ ]:
model = NMTAttn()
print(model)

Serial_in2_out2[
  Select[0,1,0,1]_in2_out4
  Parallel_in2_out2[
    Serial[
      Embedding_250100_256
      LSTM_256
      LSTM_256
    ]
    Serial[
      Serial[
        ShiftRight(1)
      ]
      Embedding_250100_256
      LSTM_256
    ]
  ]
  PrepareAttentionInput_in3_out4
  Serial_in4_out2[
    Branch_in4_out3[
      None
      Serial_in4_out2[
        _in4_out4
        Serial_in4_out2[
          Parallel_in3_out3[
            Dense_256
            Dense_256
            Dense_256
          ]
          PureAttention_in4_out2
          Dense_256
        ]
        _in2_out2
      ]
    ]
    Add_in2
  ]
  Select[0,2]_in3_out2
  LSTM_256
  LSTM_256
  Dense_250100
  LogSoftmax
]


In [ ]:
def train_task_function(train_batch_stream):
    return training.TrainTask(
        labeled_data= train_batch_stream,

        loss_layer= tl.CrossEntropyLoss(),

        optimizer= trax.optimizers.Adam(0.01),

        lr_schedule= trax.lr.warmup_and_rsqrt_decay(n_warmup_steps=1000,max_value=0.01),

        n_steps_per_checkpoint= 10

    )



In [ ]:
train_task = train_task_function(train_batch_stream)

In [ ]:
eval_task = training.EvalTask(

    labeled_data= eval_batch_stream,

    metrics=[tl.CrossEntropyLoss(), tl.Accuracy()],
)

In [ ]:
output_dir ='/content/drive/My Drive/Opus100_DIr/Model'

# remove old model if it exists. restarts training.
!rm -f ~/output_dir/model.pkl.gz

# define the training loop
training_loop = training.Loop(NMTAttn(mode='train'),
                              train_task,
                              eval_tasks=[eval_task],
                              output_dir=output_dir)

/usr/local/lib/python3.9/dist-packages/jax/_src/xla_bridge.py:658: UserWarning: jax.host_count has been renamed to jax.process_count. This alias will eventually be removed; please update your code.
  warnings.warn(


TypeError: ignored

In [ ]:
# NOTE: Execute the training loop. This will take around 11 minutes to complete.
training_loop.run(2)